**Recommendation:** In this note book we will use lightGBM, clusters and Apriori to create set of products which can be recommend to the user

**Loading Libraires**

In [1]:
import numpy as np
import pandas as pd
import math
import lightgbm as lgb
import matplotlib.pyplot as plt
import operator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import os
print(os.listdir("../input"))


['apriori2', 'apriori-segments', 'instacart-market-basket-analysis', 'apriori-seg']


**Loading Input data**

In [2]:
IDIR = '../input/instacart-market-basket-analysis/'
print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(IDIR + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])


loading prior
loading train
loading orders
loading products


**Loading Apriori and cluster data**

In [3]:
print(os.listdir("../input/apriori-segments"))
apriori = pd.read_csv('../input/apriori-segments/Apriori.csv')
user_top5_aisle_product = pd.read_csv('../input/apriori-segments/user_top5_aisle_product.csv')

['Apriori.csv', 'user_top5_aisle_product.csv']


In [4]:
apriori.head()

,Unnamed: 0,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,0,1337,7922,5,0.012986,5,0.012986,6,0.015583,1.000000,0.833333,64.173333
1,2,16668,19064,4,0.010389,5,0.012986,5,0.012986,0.800000,0.800000,61.606400
2,213,22864,3567,6,0.015583,7,0.018180,6,0.015583,0.857143,1.000000,55.005714
3,232,21640,20739,6,0.015583,7,0.018180,6,0.015583,0.857143,1.000000,55.005714
4,241,16096,42305,5,0.012986,7,0.018180,5,0.012986,0.714286,1.000000,55.005714


In [5]:
user_top5_aisle_product.head()

,Unnamed: 0,user_id,cluster,aisle1,product1,aisle2,product2,aisle3,product3,aisle4,product4,aisle5,product5
0,0,284,0,fresh fruits,Banana,fresh vegetables,Organic Yellow Onion,yogurt,Total 2% with Strawberry Lowfat Greek Strained...,packaged vegetables fruits,Organic Baby Spinach,packaged cheese,Organic Whole String Cheese
1,1,293,0,fresh fruits,Banana,fresh vegetables,Organic Yellow Onion,yogurt,Total 2% with Strawberry Lowfat Greek Strained...,packaged vegetables fruits,Organic Baby Spinach,packaged cheese,Organic Whole String Cheese
2,2,319,0,fresh fruits,Banana,fresh vegetables,Organic Yellow Onion,yogurt,Total 2% with Strawberry Lowfat Greek Strained...,packaged vegetables fruits,Organic Baby Spinach,packaged cheese,Organic Whole String Cheese
3,3,358,0,fresh fruits,Banana,fresh vegetables,Organic Yellow Onion,yogurt,Total 2% with Strawberry Lowfat Greek Strained...,packaged vegetables fruits,Organic Baby Spinach,packaged cheese,Organic Whole String Cheese
4,4,432,1,fresh vegetables,Organic Yellow Onion,fresh fruits,Banana,packaged vegetables fruits,Organic Baby Spinach,packaged cheese,Organic Whole String Cheese,yogurt,Total 2% with Strawberry Lowfat Greek Strained...


In [6]:
#Creating orders list for 2500 users
ordersTrain=orders[orders['eval_set']=='train']
print(len(ordersTrain))
train_usr = pd.DataFrame(ordersTrain['user_id'].unique(), columns =['user_id'])
train_usr_sample =  train_usr.sample(2500, random_state=42).sort_values('user_id').reset_index(drop = True)
train_usr_sample.shape

131209


(2500, 1)

**Creating Product level features**

In [7]:
prods = pd.DataFrame()
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.int32)
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
products = products.join(prods, on='product_id')
products.set_index('product_id', drop=False, inplace=True)
del prods

In [8]:
print('add order info to priors')
orders.set_index('order_id', inplace=True, drop=False)
priors = priors.join(orders, on='order_id', rsuffix='_')
priors.drop('order_id_', inplace=True, axis=1)
products.head()

add order info to priors


,product_id,aisle_id,department_id,orders,reorders,reorder_rate
product_id,,,,,,
1,1,61,19,1852.0,1136.0,0.613391
2,2,104,13,90.0,12.0,0.133333
3,3,94,7,277.0,203.0,0.732852
4,4,38,1,329.0,147.0,0.446809
5,5,5,13,15.0,9.0,0.600000


**Creating user level features**

In [9]:

usr = pd.DataFrame()
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

In [10]:
users = pd.DataFrame()
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)
users['user_max_order_num'] =  priors.groupby('user_id')['order_number'].max().astype(np.int16)
users['total_buy_max'] =  priors.groupby(['user_id','product_id'])['product_id'].count().reset_index(level = 'user_id').reset_index(drop = True).groupby('user_id').max().astype(np.int16)
users = users.join(usr) 
del usr
users['average_basket'] = (users.total_items / users.nb_orders).astype(np.float32)
users.head()

,total_items,all_products,total_distinct_items,user_max_order_num,total_buy_max,average_days_between_orders,nb_orders,average_basket
user_id,,,,,,,,
1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,10,10,19.000000,11,5.363636
2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,14,9,16.285715,15,13.000000
3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12,10,12.000000,13,6.769231
4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,5,2,17.000000,6,3.000000
5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,4,4,11.500000,5,7.400000


**Creating user product level features**

In [11]:
priors['user_product'] = priors.product_id + priors.user_id * 100000

In [12]:
d= dict()
for row in priors.itertuples():
    z = row.user_product
    if z not in d:
        d[z] = (1,
                (row.order_number, row.order_id),
                (row.order_number, row.order_id),
                row.add_to_cart_order)
    else:
        d[z] = (d[z][0] + 1,
                max(d[z][1], (row.order_number, row.order_id)),
                min(d[z][2], (row.order_number, row.order_id)),
                d[z][3] + row.add_to_cart_order)

In [13]:
userXproduct = pd.DataFrame.from_dict(d, orient='index')
del d

In [14]:
userXproduct.columns = ['nb_orders', 'last_order_id','first_order_number', 'sum_pos_in_cart']
userXproduct.nb_orders = userXproduct.nb_orders.astype(np.int16)
userXproduct.last_order_id = userXproduct.last_order_id.map(lambda x: x[1]).astype(np.int32)
userXproduct.first_order_number = userXproduct.first_order_number.map(lambda x: x[0]).astype(np.int16)
userXproduct.sum_pos_in_cart = userXproduct.sum_pos_in_cart.astype(np.int16)
print('user X product f', len(userXproduct))
del priors
userXproduct.head()

user X product f 13307953


,nb_orders,last_order_id,first_order_number,sum_pos_in_cart
20227933120,5,104690,1,10
20227928985,5,132412,1,16
20227909327,1,2,3,3
20227945918,5,2382766,2,24
20227930035,3,2382766,3,14


![](http://)**Spliting data for 2500 users**

In [15]:
train_orders = orders[orders.eval_set == 'train']
train_orders_train = train_orders[~train_orders.user_id.isin(train_usr_sample['user_id'])]
train_orders_cv = train_orders[train_orders.user_id.isin(train_usr_sample['user_id'])]
train_orders_product  = train_orders_cv.merge(train, how = 'inner', left_on = 'order_id', right_on = 'order_id')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'order_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.


In [16]:
train_orders_product.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,3261724,284,train,17,2,11,13.0,49235,1,1
1,3261724,284,train,17,2,11,13.0,35951,2,1
2,3261724,284,train,17,2,11,13.0,32579,3,1
3,3261724,284,train,17,2,11,13.0,30066,4,1
4,3261724,284,train,17,2,11,13.0,25659,5,1


In [17]:
train.set_index(['order_id', 'product_id'], inplace=True, drop=False)

**Defining a function to combine all the features **

In [18]:
def features(selected_orders, labels_given=False):
    print('build candidate list')
    order_list = []
    product_list = []
    labels = []
    i=0
    for row in selected_orders.itertuples():
        i+=1
        if i%10000 == 0: print('order row',i)
        order_id = row.order_id
        user_id = row.user_id
        user_products = users.all_products[user_id]
        product_list += user_products
        order_list += [order_id] * len(user_products)
        if labels_given:
            labels += [(order_id, product) in train.index for product in user_products]
        
    df = pd.DataFrame({'order_id':order_list, 'product_id':product_list}, dtype=np.int32)
    labels = np.array(labels, dtype=np.int8)
    del order_list
    del product_list
    
    print('user related features')
    df['user_id'] = df.order_id.map(orders.user_id)
    df['user_total_orders'] = df.user_id.map(users.nb_orders)
    df['user_total_items'] = df.user_id.map(users.total_items)
    df['total_distinct_items'] = df.user_id.map(users.total_distinct_items)
    df['user_average_days_between_orders'] = df.user_id.map(users.average_days_between_orders)
    df['user_average_basket'] =  df.user_id.map(users.average_basket)
    df['user_total_buy_max'] = df.user_id.map(users.total_buy_max).astype(np.int16)
    
    print('order related features')
    df['order_dow'] = df.order_id.map(orders.order_dow)
    df['order_hour_of_day'] = df.order_id.map(orders.order_hour_of_day)
    df['days_since_prior_order'] = df.order_id.map(orders.days_since_prior_order)
    df['days_since_ratio'] = (df.days_since_prior_order / df.user_average_days_between_orders).map(lambda x: 0 if math.isnan(x) else x).astype(np.float32)
    
    print('product related features')
    df['aisle_id'] = df.product_id.map(products.aisle_id)
    df['department_id'] = df.product_id.map(products.department_id)
    df['product_orders'] = df.product_id.map(products.orders).astype(np.int32)
    df['product_reorders'] = df.product_id.map(products.reorders)
    df['product_reorder_rate'] = df.product_id.map(products.reorder_rate).astype(np.float32)

    print('user_X_product related features')
    df['z'] = df.user_id * 100000 + df.product_id
    #df.drop(['user_id'], axis=1, inplace=True)
    df['UP_orders'] = df.z.map(userXproduct.nb_orders)
    df['UP_orders_ratio'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
    df['UP_last_order_id'] = df.z.map(userXproduct.last_order_id)
    df['UP_average_pos_in_cart'] = (df.z.map(userXproduct.sum_pos_in_cart) / df.UP_orders).astype(np.float32)
    df['UP_reorder_rate'] = ((df.UP_orders-1) / (df.user_total_orders-1).astype(np.float32))
    df['UP_orders_since_last'] = df.user_total_orders - df.UP_last_order_id.map(orders.order_number)
    df['UP_delta_hour_vs_last'] = abs(df.order_hour_of_day - df.UP_last_order_id.map(orders.order_hour_of_day)).map(lambda x: min(x, 24-x)).astype(np.int8)
    df['UP_delta_dow_vs_last'] = abs(df.order_dow - df.UP_last_order_id.map(orders.order_dow)).map(lambda x: min(x, 7-x)).astype(np.int8)
    df['UP_drop_chance'] = (df.user_total_orders - df.UP_last_order_id.map(orders.order_number)).astype(np.float)
    df['UP_chance_vs_bought'] = (df.user_total_orders - df.z.map(userXproduct.first_order_number)).astype(np.float32)
    df['UP_chance'] = (df.UP_orders - 1)/(df.user_total_orders - df.z.map(userXproduct.first_order_number)).astype(np.float32)
    df['UP_chance_ratio'] = (1/(df.user_total_orders - df.UP_last_order_id.map(orders.order_number)) - (df.UP_orders - 1)/(df.user_total_orders - df.z.map(userXproduct.first_order_number))).astype(np.float32)
    df.drop(['UP_last_order_id','z'], axis=1, inplace=True)
    #df.drop(['order_id','product_id'], axis=1)
    print(df.dtypes)
    print(df.memory_usage())
    return (df, labels)

**Getting all the features**

In [19]:
train_orders_train = train_orders[~train_orders.user_id.isin(train_usr_sample['user_id'])]
train_orders_cv = train_orders[train_orders.user_id.isin(train_usr_sample['user_id'])]

In [20]:
train_orders_cv.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
3261724,3261724,284,train,17,2,11,13.0
2692535,2692535,293,train,9,1,16,30.0
1054904,1054904,319,train,9,0,18,2.0
394279,394279,358,train,8,1,17,30.0
488785,488785,432,train,53,0,19,7.0


In [21]:
df_train, labels_train = features(train_orders_train, labels_given=True)
df_cv, labels_cv = features(train_orders_cv, labels_given=True)

build candidate list
order row 10000
order row 20000
order row 30000
order row 40000
order row 50000
order row 60000
order row 70000
order row 80000
order row 90000
order row 100000
order row 110000
order row 120000
user related features
order related features
product related features
user_X_product related features
order_id                              int32
product_id                            int32
user_id                               int32
user_total_orders                     int16
user_total_items                      int16
total_distinct_items                  int16
user_average_days_between_orders    float32
user_average_basket                 float32
user_total_buy_max                    int16
order_dow                              int8
order_hour_of_day                      int8
days_since_prior_order              float32
days_since_ratio                    float32
aisle_id                              uint8
department_id                         uint8
product_orders        

**Training light GBM **

In [22]:
features_to_use = ['user_total_orders', 'user_total_items',
       'total_distinct_items', 'user_average_days_between_orders',
       'user_average_basket', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
       'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
       'UP_average_pos_in_cart', 'UP_reorder_rate', 'UP_orders_since_last',
       'UP_delta_hour_vs_last', 'UP_delta_dow_vs_last', 'UP_drop_chance',
       'UP_chance_vs_bought', 'user_total_buy_max', 'UP_chance', 'UP_chance_ratio','aisle_id']

In [23]:
print('formating for lgb')
d_train = lgb.Dataset(df_train[features_to_use],
                      label=labels_train,
                      categorical_feature=['aisle_id', 'department_id']) 

formating for lgb


In [24]:
##Best Parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

In [25]:
print('light GBM train :-)')
bst = lgb.train(params, d_train, ROUNDS)

light GBM train :-)


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


**Function to finding products to recommend using light GBM**

In [26]:
def pred_fo (df_order,order_id):
    
    preds = bst.predict(df_order[features_to_use])
    df_order['pred'] = preds
    d = dict()
    for row in df_order.itertuples():
        if row.pred > 0.22:
            try:
                d[row.order_id] += ' ' + str(row.product_id)
            except:
                d[row.order_id] = str(row.product_id)

    for order in df_order.order_id:
        if order not in d:
            d[order] = 'None'
            
    sub = pd.DataFrame.from_dict(d, orient='index')
    
    sub.reset_index(inplace=True)
    sub.columns = ['order_id', 'products']
    
    actual = list([i.split(' ') for i in list(sub['products'])])
    predicted  = list(train_orders_product[train_orders_product['order_id'] == order_id]['product_id'])
    acc = 0
    rr = [val for val in predicted if val in actual]
    if len(rr) == 0:
        acc = 0
    else:
        acc = 1
    
    return (acc)

**Function to find accuracy of the recommended products**

In [27]:
def recommend (orderid):
    df_cv_l  = df_cv
    df_cv_l['label'] = labels_cv
    df_order = df_cv_l[df_cv_l['order_id'] == orderid]
    accuracy = pred_fo(df_order,orderid)
    
    df_ol = df_cv_l[df_cv_l['order_id'] == orderid]
    df_op = df_ol[df_ol['label'] == 1]
    products = list(train_orders_product[train_orders_product['order_id'] == orderid]['product_id'])
    user = list(df_op['user_id'].unique())
    user_products_cluster = user_top5_aisle_product[user_top5_aisle_product.user_id.isin (df_op['user_id'].unique())]
    upc_product = list(user_products_cluster['product1'])
    upc_product.append(list(user_products_cluster['product2']))
    for i in products:
        temp = apriori[apriori['item_A'] == i].nlargest(9,'lift')
        p_products = list(temp['item_B'])
        p_products.append(upc_product)
        rr = [val for val in p_products if val in products]
        if len(rr) > 0:
            accuracy = accuracy + 1
    
    return(accuracy/len(products))

**Finding accuracy **

In [28]:
orders_cv = list(train_orders_cv['order_id'])
accuracy = []
j = 0
for i in orders_cv:
    print('Orders left:', 2500 - j)
    j = j+1
    accuracy.append(recommend(i))



Orders left: 2500


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Orders left: 2499
Orders left: 2498
Orders left: 2497
Orders left: 2496
Orders left: 2495
Orders left: 2494
Orders left: 2493
Orders left: 2492
Orders left: 2491
Orders left: 2490
Orders left: 2489
Orders left: 2488
Orders left: 2487
Orders left: 2486
Orders left: 2485
Orders left: 2484
Orders left: 2483
Orders left: 2482
Orders left: 2481
Orders left: 2480
Orders left: 2479
Orders left: 2478
Orders left: 2477
Orders left: 2476
Orders left: 2475
Orders left: 2474
Orders left: 2473
Orders left: 2472
Orders left: 2471
Orders left: 2470
Orders left: 2469
Orders left: 2468
Orders left: 2467
Orders left: 2466
Orders left: 2465
Orders left: 2464
Orders left: 2463
Orders left: 2462
Orders left: 2461
Orders left: 2460
Orders left: 2459
Orders left: 2458
Orders left: 2457
Orders left: 2456
Orders left: 2455
Orders left: 2454
Orders left: 2453
Orders left: 2452
Orders left: 2451
Orders left: 2450
Orders left: 2449
Orders left: 2448
Orders left: 2447
Orders left: 2446
Orders left: 2445
Orders lef

Orders left: 2043
Orders left: 2042
Orders left: 2041
Orders left: 2040
Orders left: 2039
Orders left: 2038
Orders left: 2037
Orders left: 2036
Orders left: 2035
Orders left: 2034
Orders left: 2033
Orders left: 2032
Orders left: 2031
Orders left: 2030
Orders left: 2029
Orders left: 2028
Orders left: 2027
Orders left: 2026
Orders left: 2025
Orders left: 2024
Orders left: 2023
Orders left: 2022
Orders left: 2021
Orders left: 2020
Orders left: 2019
Orders left: 2018
Orders left: 2017
Orders left: 2016
Orders left: 2015
Orders left: 2014
Orders left: 2013
Orders left: 2012
Orders left: 2011
Orders left: 2010
Orders left: 2009
Orders left: 2008
Orders left: 2007
Orders left: 2006
Orders left: 2005
Orders left: 2004
Orders left: 2003
Orders left: 2002
Orders left: 2001
Orders left: 2000
Orders left: 1999
Orders left: 1998
Orders left: 1997
Orders left: 1996
Orders left: 1995
Orders left: 1994
Orders left: 1993
Orders left: 1992
Orders left: 1991
Orders left: 1990
Orders left: 1989
Orders lef

Orders left: 1587
Orders left: 1586
Orders left: 1585
Orders left: 1584
Orders left: 1583
Orders left: 1582
Orders left: 1581
Orders left: 1580
Orders left: 1579
Orders left: 1578
Orders left: 1577
Orders left: 1576
Orders left: 1575
Orders left: 1574
Orders left: 1573
Orders left: 1572
Orders left: 1571
Orders left: 1570
Orders left: 1569
Orders left: 1568
Orders left: 1567
Orders left: 1566
Orders left: 1565
Orders left: 1564
Orders left: 1563
Orders left: 1562
Orders left: 1561
Orders left: 1560
Orders left: 1559
Orders left: 1558
Orders left: 1557
Orders left: 1556
Orders left: 1555
Orders left: 1554
Orders left: 1553
Orders left: 1552
Orders left: 1551
Orders left: 1550
Orders left: 1549
Orders left: 1548
Orders left: 1547
Orders left: 1546
Orders left: 1545
Orders left: 1544
Orders left: 1543
Orders left: 1542
Orders left: 1541
Orders left: 1540
Orders left: 1539
Orders left: 1538
Orders left: 1537
Orders left: 1536
Orders left: 1535
Orders left: 1534
Orders left: 1533
Orders lef

Orders left: 1131
Orders left: 1130
Orders left: 1129
Orders left: 1128
Orders left: 1127
Orders left: 1126
Orders left: 1125
Orders left: 1124
Orders left: 1123
Orders left: 1122
Orders left: 1121
Orders left: 1120
Orders left: 1119
Orders left: 1118
Orders left: 1117
Orders left: 1116
Orders left: 1115
Orders left: 1114
Orders left: 1113
Orders left: 1112
Orders left: 1111
Orders left: 1110
Orders left: 1109
Orders left: 1108
Orders left: 1107
Orders left: 1106
Orders left: 1105
Orders left: 1104
Orders left: 1103
Orders left: 1102
Orders left: 1101
Orders left: 1100
Orders left: 1099
Orders left: 1098
Orders left: 1097
Orders left: 1096
Orders left: 1095
Orders left: 1094
Orders left: 1093
Orders left: 1092
Orders left: 1091
Orders left: 1090
Orders left: 1089
Orders left: 1088
Orders left: 1087
Orders left: 1086
Orders left: 1085
Orders left: 1084
Orders left: 1083
Orders left: 1082
Orders left: 1081
Orders left: 1080
Orders left: 1079
Orders left: 1078
Orders left: 1077
Orders lef

Orders left: 656
Orders left: 655
Orders left: 654
Orders left: 653
Orders left: 652
Orders left: 651
Orders left: 650
Orders left: 649
Orders left: 648
Orders left: 647
Orders left: 646
Orders left: 645
Orders left: 644
Orders left: 643
Orders left: 642
Orders left: 641
Orders left: 640
Orders left: 639
Orders left: 638
Orders left: 637
Orders left: 636
Orders left: 635
Orders left: 634
Orders left: 633
Orders left: 632
Orders left: 631
Orders left: 630
Orders left: 629
Orders left: 628
Orders left: 627
Orders left: 626
Orders left: 625
Orders left: 624
Orders left: 623
Orders left: 622
Orders left: 621
Orders left: 620
Orders left: 619
Orders left: 618
Orders left: 617
Orders left: 616
Orders left: 615
Orders left: 614
Orders left: 613
Orders left: 612
Orders left: 611
Orders left: 610
Orders left: 609
Orders left: 608
Orders left: 607
Orders left: 606
Orders left: 605
Orders left: 604
Orders left: 603
Orders left: 602
Orders left: 601
Orders left: 600
Orders left: 599
Orders left: 5

Orders left: 174
Orders left: 173
Orders left: 172
Orders left: 171
Orders left: 170
Orders left: 169
Orders left: 168
Orders left: 167
Orders left: 166
Orders left: 165
Orders left: 164
Orders left: 163
Orders left: 162
Orders left: 161
Orders left: 160
Orders left: 159
Orders left: 158
Orders left: 157
Orders left: 156
Orders left: 155
Orders left: 154
Orders left: 153
Orders left: 152
Orders left: 151
Orders left: 150
Orders left: 149
Orders left: 148
Orders left: 147
Orders left: 146
Orders left: 145
Orders left: 144
Orders left: 143
Orders left: 142
Orders left: 141
Orders left: 140
Orders left: 139
Orders left: 138
Orders left: 137
Orders left: 136
Orders left: 135
Orders left: 134
Orders left: 133
Orders left: 132
Orders left: 131
Orders left: 130
Orders left: 129
Orders left: 128
Orders left: 127
Orders left: 126
Orders left: 125
Orders left: 124
Orders left: 123
Orders left: 122
Orders left: 121
Orders left: 120
Orders left: 119
Orders left: 118
Orders left: 117
Orders left: 1

In [29]:
print('Mean accuracy across all orders:', np.mean(accuracy))

Mean accuracy across all orders: 0.09650264510601003
